
# ENEM 2023 — EDA (Univariada, Região/UF, Socioeconômicas)

Este notebook foi gerado para apoiar seu **Checkpoint**:
- Estatísticas descritivas e **gráficos univariados**
- **Comparações por Região/UF** (a partir de `CO_MUNICIPIO_PROVA`)
- **Socioeconômico**: Q001/Q002 (escolaridade dos pais), Q006 (renda), Q022/Q024/Q025
- **Correlação** (Spearman para ordinais) e tabelas exportáveis

> **Pré-requisito**: Garanta que você já executou o `prepare_enem.py` atualizado e que seu Parquet contenha as colunas derivadas `SG_UF_PROVA`, `REGIAO_NOME_PROVA`, `*_ord` (para Q001, Q002, Q006, Q022, Q024, Q025) e `NOTA_MEDIA_5`.


In [ ]:

# === Configurações ===
PARQUET_PATH = "data/interim/enem_2023.parquet"  # ajuste o caminho conforme seu projeto
EXPORT_DIR = "data/processed/eda_exports"

# === Imports ===
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path

# Regras para gráficos neste projeto (sem seaborn)
plt.rcParams.update({
    "figure.figsize": (9, 5),
})

Path(EXPORT_DIR).mkdir(parents=True, exist_ok=True)

print("OK: Imports e diretórios prontos.")


In [ ]:

# === Carregar dados ===
df = pd.read_parquet(PARQUET_PATH)
print(f"Linhas: {len(df):,}")
print("Colunas disponíveis:", sorted(df.columns.tolist())[:20], "...")

# Colunas de notas e socioeconômicas
NOTE_COLS = ["NU_NOTA_MT","NU_NOTA_LC","NU_NOTA_CH","NU_NOTA_CN","NU_NOTA_REDACAO"]
SOCIO_ORD = [c for c in ["Q001_ord","Q002_ord","Q006_ord","Q022_ord","Q024_ord","Q025_ord"] if c in df.columns]

# Verificações importantes
needed = ["NOTA_MEDIA_5","SG_UF_PROVA","REGIAO_NOME_PROVA"]
missing = [c for c in needed if c not in df.columns]
if missing:
    print("⚠️ Faltam colunas derivadas no Parquet:", missing)
else:
    print("✔️ Colunas derivadas OK:", needed)

df.head(3)


## 1) Estatísticas descritivas — notas

In [ ]:

desc_notes = df[NOTE_COLS + ["NOTA_MEDIA_5"]].describe().T
display(desc_notes)
desc_notes.to_csv(f"{EXPORT_DIR}/01_desc_notas.csv")
print("Exportado: 01_desc_notas.csv")


In [ ]:

# Histogramas — NOTA_MEDIA_5 e provas individuais
for col in ["NOTA_MEDIA_5"] + NOTE_COLS:
    if col in df.columns:
        plt.figure()
        df[col].plot(kind='hist', bins=40, edgecolor='black')
        plt.title(f'Histograma — {col}')
        plt.xlabel(col)
        plt.ylabel('Frequência')
        plt.show()


## 2) Comparações por Região e UF (local da prova)

In [ ]:

# Médias por Região
if "REGIAO_NOME_PROVA" in df.columns:
    mean_by_region = (df.groupby("REGIAO_NOME_PROVA")["NOTA_MEDIA_5"]
                        .mean().sort_values(ascending=False))
    display(mean_by_region)
    mean_by_region.to_csv(f"{EXPORT_DIR}/02_media_por_regiao.csv")
    print("Exportado: 02_media_por_regiao.csv")

    # Gráfico de barras — Região
    plt.figure()
    mean_by_region.plot(kind="bar", edgecolor="black")
    plt.title("NOTA_MEDIA_5 — média por Região (local da prova)")
    plt.xlabel("Região")
    plt.ylabel("Média da nota")
    plt.xticks(rotation=45)
    plt.show()
else:
    print("REGIAO_NOME_PROVA não disponível.")
    
# Médias por UF
if "SG_UF_PROVA" in df.columns:
    mean_by_uf = (df.groupby("SG_UF_PROVA")["NOTA_MEDIA_5"]
                    .mean().sort_values(ascending=False))
    display(mean_by_uf.head(10))
    mean_by_uf.to_csv(f"{EXPORT_DIR}/03_media_por_uf.csv")
    print("Exportado: 03_media_por_uf.csv")

    # Gráfico de barras — top 15 UFs por média
    plt.figure()
    mean_by_uf.head(15).plot(kind="bar", edgecolor="black")
    plt.title("NOTA_MEDIA_5 — Top 15 UFs por média (local da prova)")
    plt.xlabel("UF")
    plt.ylabel("Média da nota")
    plt.xticks(rotation=45)
    plt.show()
else:
    print("SG_UF_PROVA não disponível.")


## 3) Tipo de escola (TP_ESCOLA) × nota

In [ ]:

if "TP_ESCOLA" in df.columns:
    mapa_tp = {1: "Não Respondeu", 2: "Pública", 3: "Privada"}
    df["TP_ESCOLA_rot"] = df["TP_ESCOLA"].map(mapa_tp)
    mean_escola = df.groupby("TP_ESCOLA_rot")["NOTA_MEDIA_5"].mean().sort_values(ascending=False)
    display(mean_escola)
    mean_escola.to_csv(f"{EXPORT_DIR}/04_media_por_tp_escola.csv")
    print("Exportado: 04_media_por_tp_escola.csv")

    plt.figure()
    mean_escola.plot(kind="bar", edgecolor="black")
    plt.title("NOTA_MEDIA_5 — média por Tipo de Escola")
    plt.xlabel("Tipo de Escola")
    plt.ylabel("Média da nota")
    plt.xticks(rotation=0)
    plt.show()
else:
    print("TP_ESCOLA não disponível no Parquet.")


## 4) Socioeconômico — correlação (Spearman)

In [ ]:

from scipy.stats import spearmanr

corr_rows = []
for col in [c for c in ["Q001_ord","Q002_ord","Q006_ord","Q022_ord","Q024_ord","Q025_ord"] if c in df.columns]:
    s = df[[col, "NOTA_MEDIA_5"]].dropna()
    if len(s) > 0:
        r, p = spearmanr(s[col], s["NOTA_MEDIA_5"])
        corr_rows.append({"variavel": col, "spearman_r": r, "p_value": p, "n": len(s)})

corr_df = pd.DataFrame(corr_rows).sort_values("spearman_r", ascending=False)
display(corr_df)
corr_df.to_csv(f"{EXPORT_DIR}/05_correlacoes_socio_spearman.csv", index=False)
print("Exportado: 05_correlacoes_socio_spearman.csv")


## 5) Relação Renda (Q006_ord) × Nota — tendência por categorias

In [ ]:

if "Q006_ord" in df.columns:
    grp = df.groupby("Q006_ord")["NOTA_MEDIA_5"].agg(["count","mean","median"]).reset_index()
    display(grp.head(10))
    grp.to_csv(f"{EXPORT_DIR}/06_q006_vs_nota.csv", index=False)
    print("Exportado: 06_q006_vs_nota.csv")

    plt.figure()
    plt.plot(grp["Q006_ord"], grp["mean"], marker="o")
    plt.title("Tendência da NOTA_MEDIA_5 por categoria de Renda (Q006_ord)")
    plt.xlabel("Q006_ord (renda — ordinal)")
    plt.ylabel("Média da NOTA_MEDIA_5")
    plt.grid(True, linestyle="--", alpha=0.4)
    plt.show()
else:
    print("Q006_ord não disponível.")


## 6) Tabelas cruzadas — Região × Tipo de Escola (média de nota)

In [ ]:

if set(["REGIAO_NOME_PROVA","TP_ESCOLA"]).issubset(df.columns):
    mapa_tp = {1: "Não Respondeu", 2: "Pública", 3: "Privada"}
    df["TP_ESCOLA_rot"] = df["TP_ESCOLA"].map(mapa_tp)
    pivot_reg_escola = pd.pivot_table(
        df, values="NOTA_MEDIA_5",
        index="REGIAO_NOME_PROVA", columns="TP_ESCOLA_rot",
        aggfunc="mean"
    )
    display(pivot_reg_escola)
    pivot_reg_escola.to_csv(f"{EXPORT_DIR}/07_pivot_regiao_escola.csv")
    print("Exportado: 07_pivot_regiao_escola.csv")
else:
    print("Região e/ou TP_ESCOLA indisponíveis.")


## 7) Municípios com mais participantes (amostra top 20)

In [ ]:

if "NO_MUNICIPIO_PROVA" in df.columns:
    top_mun = (df["NO_MUNICIPIO_PROVA"]
               .value_counts().head(20)
               .rename_axis("municipio").reset_index(name="contagem"))
    display(top_mun)
    top_mun.to_csv(f"{EXPORT_DIR}/08_top20_municipios.csv", index=False)
    print("Exportado: 08_top20_municipios.csv")
else:
    print("NO_MUNICIPIO_PROVA não disponível.")


## 8) Exportação de amostras / subset para validação

In [ ]:

sample = df.sample(min(10000, len(df)), random_state=42)  # amostra para depuração local
sample_path = f"{EXPORT_DIR}/09_sample_10k.csv"
sample.to_csv(sample_path, index=False)
print("Exportado:", sample_path)



## 9) Anotações para o relatório
- Interprete os **padrões regionais**: diferenças de média entre regiões e UFs podem refletir fatores socioeconômicos e de oferta escolar.
- Verifique a **tendência da renda (Q006_ord)**: confirme se o padrão aparente de linearidade é estável em todas as regiões (faça o mesmo gráfico segmentado por `REGIAO_NOME_PROVA` se necessário).
- Compare **tipo de escola** × **nota** e discuta potenciais vieses de seleção.
- Utilize **correlações de Spearman** para ordinais e discuta magnitude e significância (p-value).
- Pontue **limitações**: usamos **local da prova** como proxy de localização; não é residência.
